<a href="https://colab.research.google.com/github/dtoralg/INESDI_Data-Science_ML_IA/blob/main/%5B05%5D%20-%20Arboles%20de%20decision/%5B05%5D_Evaluacion_y_optimizacion_sencillo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluación y optimización

In [2]:
# =========================================================
# 0) IMPORTS Y DATOS (dataset binario sencillo)
# =========================================================
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Datos reales y pequeñitos: benigno vs maligno (clasificación binaria)
X, y = load_breast_cancer(return_X_y=True)

# =========================================================
# 1) SPLIT: TRAIN / VALIDACIÓN+TEST
#    (keep it simple: hacemos un holdout con test; la validación la haremos vía CV)
# =========================================================
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
# - stratify mantiene proporciones de clases (estratificado)  ← recomendado con clases desbalanceadas
#   (equivale a la "stratified cross-validation" de las diapositivas)  📌

# =========================================================
# 2) PIPELINE BASE: ESCALADO + REGRESIÓN LOGÍSTICA
#    - Escalar ayuda a la regularización a "tratar" a todas las variables por igual.
#    - Usamos solver "saga" para permitir L1 o L2.
# =========================================================
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, solver="saga"))
])

# =========================================================
# 3) MÉTRICA OBJETIVO (elegimos F1 para equilibrar precision/recall)
#    - Podrías usar accuracy, roc_auc, etc. según el problema.
# =========================================================
scoring = "f1"

# =========================================================
# 4) GRID SEARCH (prueba TODAS las combinaciones)
#    - Hiperparámetros: tipo de regularización y su intensidad (C)
#    - K-Fold estratificado para validar (cv=5)
# =========================================================
param_grid = {
    "clf__penalty": ["l1", "l2"],      # L1 (lasso) = puede anular coeficientes; L2 (ridge) = encoge sin anular
    "clf__C": [0.01, 0.1, 1, 10, 100]  # C pequeño => MÁS regularización; C grande => MENOS
}
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=scoring,
    cv=cv5,
    n_jobs=-1
)
grid.fit(X_tr, y_tr)

print("GRID - Mejor combinación:", grid.best_params_)
print(f"GRID - Mejor F1 (CV):     {grid.best_score_:.3f}")

# =========================================================
# 5) RANDOM SEARCH (prueba combinaciones AL AZAR en un rango)
#    - Suele encontrar buenas soluciones gastando menos tiempo que Grid.
# =========================================================
param_dist = {
    "clf__penalty": ["l1", "l2"],
    "clf__C": np.logspace(-3, 3, 100)  # muestreamos C en un rango amplio (10^-3 a 10^3)
}
rand = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_dist,
    n_iter=20,                # nº de combinaciones aleatorias a probar (rápido)
    scoring=scoring,
    cv=cv5,
    n_jobs=-1,
    random_state=42
)
rand.fit(X_tr, y_tr)

print("RANDOM - Mejor combinación:", rand.best_params_)
print(f"RANDOM - Mejor F1 (CV):   {rand.best_score_:.3f}")

# =========================================================
# 6) EVALUACIÓN EN TEST (examen final)
#    - ¡Siempre evalúa el MEJOR modelo en datos NO vistos!
# =========================================================
best = grid if grid.best_score_ >= rand.best_score_ else rand
y_pred = best.best_estimator_.predict(X_te)

cm = confusion_matrix(y_te, y_pred)
acc  = accuracy_score(y_te, y_pred)
prec = precision_score(y_te, y_pred)
rec  = recall_score(y_te, y_pred)
f1   = f1_score(y_te, y_pred)
spec = cm[0,0] / (cm[0,0] + cm[0,1])  # Specificity = TN / (TN + FP)

print("\n=== RESULTADOS EN TEST ===")
print("Matriz de confusión:\n", cm)
print(f"Accuracy:    {acc:.3f}")
print(f"Precision:   {prec:.3f}")
print(f"Recall:      {rec:.3f}")
print(f"F1-score:    {f1:.3f}")
print(f"Specificity: {spec:.3f}")

# =========================================================
# 7) (OPCIONAL) EARLY STOPPING con XGBoost para ver el "freno" al overfitting
#    - Requiere xgboost instalado. Si no lo tienes, comenta esta sección.
#    - Se detiene cuando no mejora en el set de validación durante 'n' rondas.
# =========================================================
# from xgboost import XGBClassifier
# xgb = XGBClassifier(
#     n_estimators=1000, learning_rate=0.05, max_depth=3, reg_lambda=1.0,
#     eval_metric="logloss", random_state=42
# )
# xgb.fit(
#     X_tr, y_tr,
#     eval_set=[(X_tr, y_tr), (X_te, y_te)],
#     early_stopping_rounds=50,  # si no mejora en 50 rondas, se para
#     verbose=False
# )
# print("\nXGBoost - Árboles realmente usados (best_iteration):", xgb.best_iteration)


GRID - Mejor combinación: {'clf__C': 0.1, 'clf__penalty': 'l2'}
GRID - Mejor F1 (CV):     0.986
RANDOM - Mejor combinación: {'clf__penalty': 'l2', 'clf__C': np.float64(0.11497569953977356)}
RANDOM - Mejor F1 (CV):   0.988

=== RESULTADOS EN TEST ===
Matriz de confusión:
 [[40  2]
 [ 1 71]]
Accuracy:    0.974
Precision:   0.973
Recall:      0.986
F1-score:    0.979
Specificity: 0.952


Cómo este ejemplo cubrimos “todas las casuísticas” del tema

Hiperparámetros & Overfitting: regulas la complejidad con penalty (L1/L2) y C (intensidad). L1 puede “apagar” variables; L2 suaviza pesos. Esto ataca el overfitting como “freno” del modelo.

Máster de FP en BA e IA - Slide…

Regularización: L1/L2 exactamente como en las diapositivas (analogías lasso/ridge).

Máster de FP en BA e IA - Slide…

Validación:

Holdout (train/test) para examen final.

K-Fold estratificado (cv=5) dentro de Grid/Random Search para seleccionar hiperparámetros sin fuga de información.

Máster de FP en BA e IA - Slide…

Búsqueda de hiperparámetros:

Grid Search (exhaustivo, más lento, “garantiza” revisar la malla).

Random Search (más rápido, buena calidad en menos tiempo).

(Mencionada la Bayesiana como tercera alternativa en clase).

Máster de FP en BA e IA - Slide…

Métricas y matriz de confusión: imprimimos accuracy, precision, recall, F1, specificity y la confusion matrix (TN, FP, FN, TP). Úsalas según el coste de errores en tu caso.

Máster de FP en BA e IA - Slide…

Early stopping (opcional): con XGBoost mostramos el corte automático cuando deja de mejorar en validación, evitando sobreentrenar.

Máster de FP en BA e IA - Slide…